# **Homework 1: COVID-19 Cases Prediction (Regression)**

Objectives:
* Solve a regression problem with deep neural networks (DNN).
* Understand basic DNN training tips.
* Familiarize yourself with PyTorch.

If you have any questions, please contact the TAs via TA hours, NTU COOL, or email to mlta-2023-spring@googlegroups.com

In [1]:
!nvidia-smi

Sat Apr  6 21:34:50 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 537.58                 Driver Version: 537.58       CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 4090      WDDM  | 00000000:01:00.0 Off |                  Off |
|  0%   35C    P8              16W / 450W |   1087MiB / 24564MiB |      2%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# Download data
If the Google Drive links below do not work, you can use the dropbox link below or download data from [Kaggle](https://www.kaggle.com/t/a339b77fa5214978bfb8dde62d3151fe), and upload data manually to the workspace.

In [2]:
# google drive link
# !pip install gdown
# !gdown --id '1BjXalPZxq9mybPKNjF3h5L3NcF7XKTS-' --output covid_train.csv
# !gdown --id '1B55t74Jg2E5FCsKCsUEkPKIuqaY7UIi1' --output covid_test.csv

# dropbox link
# !wget -O covid_train.csv https://www.dropbox.com/s/lmy1riadzoy0ahw/covid.train.csv?dl=0
# !wget -O covid_test.csv https://www.dropbox.com/s/zalbw42lu4nmhr2/covid.test.csv?dl=0

# Import packages

In [3]:
# Numerical Operations
import math
import numpy as np
from numpy import ndarray

# Reading/Writing Data
import pandas as pd
import os
import csv

# For Progress Bar
from tqdm import tqdm

# Pytorch
import torch 
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split

# For plotting learning curve
from torch.utils.tensorboard import SummaryWriter

# Some Utility Functions

You do not need to modify this part.

In [4]:
def same_seed(seed): 
    '''Fixes random number generator seeds for reproducibility.'''
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

def train_valid_split(data_set, valid_ratio, seed):
    '''Split provided training data into training set and validation set'''
    valid_set_size = int(valid_ratio * len(data_set)) 
    train_set_size = len(data_set) - valid_set_size
    train_set, valid_set = random_split(data_set, [train_set_size, valid_set_size], generator=torch.Generator().manual_seed(seed))
    return np.array(train_set), np.array(valid_set)

def predict(test_loader, model, device):
    model.eval() # Set your model to evaluation mode.
    preds = []
    for x in tqdm(test_loader):
        x = x.to(device)                        
        with torch.no_grad():                   
            pred = model(x)                     
            preds.append(pred.detach().cpu())   
    preds = torch.cat(preds, dim=0).numpy()  
    return preds

# Dataset

In [5]:
class COVID19Dataset(Dataset):
    '''
    x: Features.
    y: Targets, if none, do prediction.
    '''
    def __init__(self, x, y=None):
        if y is None:
            self.y = y
        else:
            self.y = torch.FloatTensor(y)
        self.x = torch.FloatTensor(x)

    def __getitem__(self, idx):
        if self.y is None:
            return self.x[idx]
        else:
            return self.x[idx], self.y[idx]

    def __len__(self):
        return len(self.x)

# Neural Network Model
Try out different model architectures by modifying the class below.

In [6]:
class My_Model(nn.Module):
    def __init__(self, input_dim):
        super(My_Model, self).__init__()
        # TODO: modify model's structure, be aware of dimensions. 
        self.layers = nn.Sequential(
            nn.Linear(input_dim, 16),
            nn.ReLU(),
            nn.Linear(16, 8),
            nn.ReLU(),
            nn.Linear(8, 1)
        )

    def forward(self, x):
        x = self.layers(x)
        x = x.squeeze(1) # (B, 1) -> (B)
        return x

# Feature Selection
Choose features you deem useful by modifying the function below.

In [7]:
def select_feat(train_data: ndarray, valid_data: ndarray, test_data: ndarray, select_all=True):
    '''Selects useful features to perform regression'''
    y_train, y_valid = train_data[:,-1], valid_data[:,-1]
    raw_x_train, raw_x_valid, raw_x_test = train_data[:,:-1], valid_data[:,:-1], test_data

    if select_all:
        feat_idx = list(range(raw_x_train.shape[1]))
    else:
        # feat_idx = list(range(35, raw_x_train.shape[1])) # TODO: Select suitable feature columns.
        feat_idx = [34, 36, 51, 52, 54, 70, 72, 69]
        
    return raw_x_train[:,feat_idx], raw_x_valid[:,feat_idx], raw_x_test[:,feat_idx], y_train, y_valid

# Training Loop

In [8]:
def trainer(train_loader: DataLoader, valid_loader: DataLoader, model: My_Model, config, device):

    criterion = nn.MSELoss(reduction='mean') # Define your loss function, do not modify this.

    # Define your optimization algorithm. 
    # TODO: Please check https://pytorch.org/docs/stable/optim.html to get more available algorithms.
    # TODO: L2 regularization (optimizer(weight decay...) or implement by your self).
    # optimizer = torch.optim.SGD(model.parameters(), lr=config['learning_rate'], momentum=0.7, weight_decay=1e-8)
    optimizer = torch.optim.Adam(model.parameters(), lr=config['learning_rate'], weight_decay=1e-8)
    writer = SummaryWriter() # Writer of tensoboard.

    if not os.path.isdir('./models'):
        os.mkdir('./models') # Create directory of saving models.

    n_epochs, best_loss, step, early_stop_count = config['n_epochs'], math.inf, 0, 0

    for epoch in range(n_epochs):
        model.train() # Set your model to train mode.
        loss_record = []

        # tqdm is a package to visualize your training progress.
        train_pbar = tqdm(train_loader, position=0, leave=True)

        for x, y in train_pbar:
            optimizer.zero_grad()               # Set gradient to zero.
            x, y = x.to(device), y.to(device)   # Move your data to device. 
            pred = model(x)             
            loss = criterion(pred, y)
            loss.backward()                     # Compute gradient(backpropagation).
            optimizer.step()                    # Update parameters.
            step += 1
            loss_record.append(loss.detach().item())
            
            # Display current epoch number and loss on tqdm progress bar.
            train_pbar.set_description(f'Epoch [{epoch+1}/{n_epochs}]')
            train_pbar.set_postfix({'loss': loss.detach().item()})

        mean_train_loss = sum(loss_record)/len(loss_record)
        writer.add_scalar('Loss/train', mean_train_loss, step)

        model.eval() # Set your model to evaluation mode.
        loss_record = []
        for x, y in valid_loader:
            x, y = x.to(device), y.to(device) # 暂时移除，排查 CUDA 模式运行慢的问题
            with torch.no_grad():
                pred = model(x)
                loss = criterion(pred, y)

            loss_record.append(loss.item())
            
        mean_valid_loss = sum(loss_record)/len(loss_record)
        print(f'Epoch [{epoch+1}/{n_epochs}]: Train loss: {mean_train_loss:.4f}, Valid loss: {mean_valid_loss:.4f}')
        writer.add_scalar('Loss/valid', mean_valid_loss, step)

        if mean_valid_loss < best_loss:
            best_loss = mean_valid_loss
            torch.save(model.state_dict(), config['save_path']) # Save your best model
            print('Saving model with loss {:.3f}...'.format(best_loss))
            early_stop_count = 0
        else: 
            early_stop_count += 1

        if early_stop_count >= config['early_stop']:
            print('\nModel is not improving, so we halt the training session.')
            return

# Configurations
`config` contains hyper-parameters for training and the path to save your model.

In [9]:
# device = 'cuda' if torch.cuda.is_available() else 'cpu'
device = 'cpu'
# device = 'cuda'
config = {
    'seed': 5201314,      # Your seed number, you can pick your lucky number. :)
    'select_all': False,   # Whether to use all features.
    'valid_ratio': 0.2,   # validation_size = train_size * valid_ratio
    'n_epochs': 5000,     # Number of epochs.            
    'batch_size': 256, 
    'learning_rate': 1e-5,              
    'early_stop': 600,    # If model has not improved for this many consecutive epochs, stop training.     
    'save_path': './models/model.ckpt'  # Your model will be saved here.
}

# Dataloader
Read data from files and set up training, validation, and testing sets. You do not need to modify this part.

In [10]:
# Set seed for reproducibility
same_seed(config['seed'])


# train_data size: 3009 x 89 (35 states + 18 features x 3 days) 
# test_data size: 997 x 88 (without last day's positive rate)
train_data, test_data = pd.read_csv('./data/covid_train.csv').values, pd.read_csv('./data/covid_test.csv').values
train_data, valid_data = train_valid_split(train_data, config['valid_ratio'], config['seed'])

# Print out the data size.
print(f"""train_data size: {train_data.shape} 
valid_data size: {valid_data.shape} 
test_data size: {test_data.shape}""")

# Select features
x_train, x_valid, x_test, y_train, y_valid = select_feat(train_data, valid_data, test_data, config['select_all'])

# Print out the number of features.
print(f'number of features: {x_train.shape[1]}')

train_dataset, valid_dataset, test_dataset = COVID19Dataset(x_train, y_train), \
                                            COVID19Dataset(x_valid, y_valid), \
                                            COVID19Dataset(x_test)
# train_dataset.x.to(device)
# train_dataset.y.to(device)
# valid_dataset.x.to(device)
# valid_dataset.y.to(device)
# test_dataset.x.to(device)

# Pytorch data loader loads pytorch dataset into batches.
train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True,pin_memory_device='cuda', pin_memory=True)#, pin_memory_device='cuda', pin_memory=True
valid_loader = DataLoader(valid_dataset, batch_size=config['batch_size'], shuffle=True,pin_memory_device='cuda', pin_memory=True)#, pin_memory=True
test_loader = DataLoader(test_dataset, batch_size=config['batch_size'], shuffle=False,pin_memory_device='cuda', pin_memory=True)#, pin_memory=True

train_data size: (2408, 89) 
valid_data size: (601, 89) 
test_data size: (997, 88)
number of features: 8


# Start training!
Macbook Air M1, CPU 模式，6m 11.3s
TODO：尝试使用 MPS

## 4090+i5-13600kf, CPU 模式，2m 29s
## 4090+i5-13600kf, CUDA 模式，3m 7s/5m25s
## 4090+i5-13600kf, CUDA 模式，pin_memory_device='cuda', pin_memory=True：2m 49s
## 4090+i5-13600kf, CUDA 模式，无（pin_memory_device='cuda', pin_memory=True）：3 m 15 s

## 4090+i5-13600kf, CUDA 模式
* 4m 17s，增加  [8,4] 的 layer，效果很差（kaggle score 18+）
* 3m 48s，使用 https://www.kaggle.com/code/bayes2003/ml2023spring-hw1-public-0-79 中的特征值，效果提升（katgle score 0.89+）

## 4090+i5-13600kf, CPU 模式
* 1m 3s，kaggle score: 1.2
  * 使用 https://www.kaggle.com/code/bayes2003/ml2023spring-hw1-public-0-79 中的特征值
  * weight_decay=3
* 1m 22s，katgle score: 1.0
  * 使用 https://www.kaggle.com/code/bayes2003/ml2023spring-hw1-public-0-79 中的特征值
  * weight_decay=1
* 2m 38s，katgle score: 0.89
  * 使用 https://www.kaggle.com/code/bayes2003/ml2023spring-hw1-public-0-79 中的特征值
  * weight_decay=1e-8
* 17s，katgle score: 1.27
  * 使用 https://www.kaggle.com/code/bayes2003/ml2023spring-hw1-public-0-79 中的特征值
  * weight_decay=1e-8
  * epoch=600
* 27s，katgle score: 1.17
  * 使用 https://www.kaggle.com/code/bayes2003/ml2023spring-hw1-public-0-79 中的特征值
  * weight_decay=1e-8
  * epoch=1000
* 2m 59s，katgle score: 1.13
  * 使用 https://www.kaggle.com/code/bayes2003/ml2023spring-hw1-public-0-79 中的特征值
  * weight_decay=1e-8
  * opti Adam

In [11]:
model = My_Model(input_dim=x_train.shape[1]).to(device) # put your model and data on the same computation device.
trainer(train_loader, valid_loader, model, config, device)

Epoch [1/5000]: 100%|██████████| 10/10 [00:01<00:00,  8.81it/s, loss=309]


Epoch [1/5000]: Train loss: 357.9817, Valid loss: 365.1763
Saving model with loss 365.176...


Epoch [2/5000]: 100%|██████████| 10/10 [00:00<00:00, 454.55it/s, loss=426]


Epoch [2/5000]: Train loss: 364.7136, Valid loss: 353.8948
Saving model with loss 353.895...


Epoch [3/5000]: 100%|██████████| 10/10 [00:00<00:00, 499.99it/s, loss=337]


...

Epoch [4998/5000]: Train loss: 1.1641, Valid loss: 1.1603


Epoch [4999/5000]: 100%|██████████| 10/10 [00:00<00:00, 476.18it/s, loss=0.888]


Epoch [4999/5000]: Train loss: 1.1500, Valid loss: 1.1781


Epoch [5000/5000]: 100%|██████████| 10/10 [00:00<00:00, 500.01it/s, loss=1.1]

Epoch [5000/5000]: Train loss: 1.1628, Valid loss: 1.1074


# Testing
The predictions of your model on testing set will be stored at `pred.csv`.

In [12]:
def save_pred(preds, file):
    ''' Save predictions to specified file '''
    with open(file, 'w') as fp:
        writer = csv.writer(fp)
        writer.writerow(['id', 'tested_positive'])
        for i, p in enumerate(preds):
            writer.writerow([i, p])

model = My_Model(input_dim=x_train.shape[1]).to(device)
model.load_state_dict(torch.load(config['save_path']))
preds = predict(test_loader, model, device) 
save_pred(preds, 'pred.csv')         

100%|██████████| 4/4 [00:00<00:00, 887.31it/s]


# Download

Run this block to download the `pred.csv` by clicking.

In [13]:
from IPython.display import FileLink
FileLink(r'pred.csv')

c:\Users\a\Documents\VSCodeRepos\DeepLearning-HW\HW1\pred.csv

# Reference
This notebook uses code written by Heng-Jui Chang @ NTUEE (https://github.com/ga642381/ML2021-Spring/blob/main/HW01/HW01.ipynb)